# Nubes de palabras

## Carga de librerías necesarias

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from collections import Counter
from ast import literal_eval

## Funciones

In [2]:
def save_cluster_wordclouds(df, output_dir,
                            width=800, height=400,
                            background_color='white',
                            colormap='viridis',
                            max_words=None):
    """
    Genera y guarda una word‑cloud por cada cluster del DataFrame,
    acumulando la frecuencia de cada palabra única.

    Parámetros
    ----------
    df : pandas.DataFrame
        Debe tener las columnas ['cluster', 'cluster_words_list']
        donde 'cluster_words_list' es una lista de strings.
    output_dir : str
        Carpeta donde se guardarán las imágenes. Se crea si no existe.
    width, height : int
        Tamaño de la imagen en píxels.
    background_color : str
        Color de fondo.
    colormap : str
        Colormap de Matplotlib.
    max_words : int | None
        Número máximo de palabras a mostrar (None = todas).
    """
    os.makedirs(output_dir, exist_ok=True)

    for cluster in df['cluster'].unique():
        # 1) Reunir todas las palabras
        lists = df.loc[df['cluster'] == cluster, 'cluster_words_list']
        
        # Contar cada aparición, incluso duplicados en la misma lista:
        all_words = [w for lst in lists for w in lst]
        
        # 2) Acumular frecuencias
        freqs = Counter(all_words)

        # 3) Generar la nube a partir de esas frecuencias
        wc = WordCloud(
            width=width,
            height=height,
            background_color=background_color,
            max_words=max_words or len(freqs),
            colormap=colormap
        ).generate_from_frequencies(freqs)

        # 4) Dibujar y guardar
        plt.figure(figsize=(width/100, height/100))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')
        fname = os.path.join(output_dir, f'cluster_{cluster}.png')
        plt.savefig(fname, bbox_inches='tight')
        plt.close()

    n = len(df['cluster'].unique())
    print(f"Guardadas {n} word‑clouds en '{output_dir}'")

## Carga de datos

In [3]:
df_macrot = pd.read_csv("../data/macro_topics.csv")
df_macrot.head().iloc[:, 1:]

,tid,n_messages,description,keywords,representative_docs,theme,rep_docs_norm,representative_docs_str,macrot_id,cluster,cluster_words,cluster_words_list
0,0,111,0_vacunamos_novavax_fauci_reacciones,"['vacunamos', 'novavax', 'fauci', 'reacciones'...",['estracto de la agencia europea del medicamen...,Conspiracy theories and misinformation about C...,['estracto de la agencia europea del medicamen...,estracto de la agencia europea del medicamento...,111,0,deaths | incl | desconocido | casos | disorders,"['deaths', 'incl', 'desconocido', 'casos', 'di..."
1,1,106,1_encontrado_oficialialistas_mujer_desconectado,"['encontrado', 'oficialialistas', 'mujer', 'de...","['los viajes espaciales, la creacion de una pa...",The theme revolves around conspiracy theories ...,['los viajes espaciales la creacion de una pan...,los viajes espaciales la creacion de una pande...,-1,-1,NaN,[]
2,2,85,2_expreso0demedianoche_directo_anonimo_descarg...,"['expreso0demedianoche', 'directo', 'anonimo',...",['el expreso de media noche directo del 24/02 ...,Unauthorized use of a popular podcast's name a...,['el expreso de media noche directo del 24 02 ...,el expreso de media noche directo del 24 02 21...,-1,-1,NaN,[]
3,3,65,3_despiertos_actuamos_inocencia_hijos,"['despiertos', 'actuamos', 'inocencia', 'hijos...",['da mucho asco ver ciertos personajes que van...,"Critique of egoism, superficiality, and lack o...",['da mucho asco ver ciertos personajes que van...,da mucho asco ver ciertos personajes que van d...,80,0,doktor | esra | medical | gmc | dr,"['doktor', 'esra', 'medical', 'gmc', 'dr']"
4,4,63,4_protestas_republica_belga_cerrados,"['protestas', 'republica', 'belga', 'cerrados'...",['la gente se defiende: suiza fuerza un refere...,Public discontent and resistance against gover...,['la gente se defiende suiza fuerza un referen...,la gente se defiende suiza fuerza un referendu...,64,8,francia | france | macron | sam | french,"['francia', 'france', 'macron', 'sam', 'french']"


In [4]:
df_macrot.shape

(21614, 13)

Se descarta el ruido.

In [5]:
df_macrot = df_macrot[df_macrot["cluster"] != -1]
df_macrot.shape

(9542, 13)

In [6]:
df_macrot["cluster_words_list"] = df_macrot["cluster_words_list"].apply(literal_eval)

## Generación de nubes de palabras

In [7]:
save_cluster_wordclouds(df_macrot, "../outputs/nubes")

Guardadas 22 word‑clouds en '../outputs/nubes'
